In [1]:
(ql:quickload :cytoscape-clj)

To load "cytoscape-clj":
  Load 1 ASDF system:
    cytoscape-clj


(:CYTOSCAPE-CLJ)


; Loading "cytoscape-clj"



In [2]:
(defclass editor ()
  ((stack
     :accessor stack
     :initform (make-array 32 :adjustable t :fill-pointer 0))
   (stack-index
     :accessor stack-index
     :initform 0)
   (cyto
     :accessor cyto
     :initform (make-instance 'cytoscape:cytoscape-widget
                              :graph-layouts (list (make-instance 'cytoscape:cose-layout))
                              :graph-style "edge { target-arrow-shape: triangle; curve-style: bezier; }"))
   (undo-command
     :accessor undo-command
     :initform (make-instance 'cytoscape:menu-command
                              :content "<span class='fa fa-undo fa-2x'></span>"))
   (redo-command
     :accessor redo-command
     :initform (make-instance 'cytoscape:menu-command
                              :content "<span class='fa fa-redo fa-2x'></span>"))
   (add-node-command
     :accessor add-node-command
     :initform (make-instance 'cytoscape:menu-command
                              :content "<span class='fa fa-plus-circle fa-2x'></span>"))
   (add-edge-command
     :accessor add-edge-command
     :initform (make-instance 'cytoscape:menu-command
                              :content "<span class='fa fa-arrow-right fa-2x'></span>"))
   (delete-node-command
     :accessor delete-node-command
     :initform (make-instance 'cytoscape:menu-command
                              :content "<span class='fa fa-trash fa-2x'></span>"))
   (delete-edge-command
     :accessor delete-edge-command
     :initform (make-instance 'cytoscape:menu-command
                              :content "<span class='fa fa-trash fa-2x'></span>"))
   (delete-elements-command
     :accessor delete-elements-command
     :initform (make-instance 'cytoscape:menu-command
                              :content "<span class='fa fa-trash fa-2x'></span>"))
   (reverse-edge-command
     :accessor reverse-edge-command
     :initform (make-instance 'cytoscape:menu-command
                              :content "<span class='fa fa-exchange-alt fa-2x'></span>"))
   (edge-source
     :accessor edge-source
     :initform nil)))


(defun update-commands (instance)
  (with-slots (delete-elements-command undo-command redo-command stack-index stack cyto)
              instance
    (setf (cytoscape:enabled delete-elements-command) (some (lambda (element) 
                                                              (and (cytoscape:selected element) 
                                                                   (not (cytoscape:removed element)))) 
                                                            (cytoscape:elements cyto)))
    (setf (cytoscape:enabled undo-command) (< 0 stack-index))
    (setf (cytoscape:enabled redo-command) (< stack-index (length stack)))))


(defun undo (instance)
  (with-slots (stack-index stack cyto)
              instance
    (when (< 0 stack-index)
      (decf stack-index)
      (dolist (element (cytoscape:elements cyto))
        (let ((pair (assoc (cdr (assoc "id" (cytoscape:data element) :test #'string=)) (elt stack stack-index) :test #'string=)))
          (when pair
            (setf (cytoscape:removed element) (cdr pair)))))
      (update-commands instance))))


(defun redo (instance)
  (with-slots (stack-index stack cyto)
              instance
    (when (< stack-index (length stack))
      (dolist (element (cytoscape:elements cyto))
        (let ((pair (assoc (cdr (assoc "id" (cytoscape:data element) :test #'string=)) (elt stack stack-index) :test #'string=)))
          (when pair
            (setf (cytoscape:removed element) (not (cdr pair))))))
      (incf stack-index)
      (update-commands instance))))


(defun add-node (instance)
  (with-slots (stack-index stack cyto)
              instance
    (let* ((new-id (jupyter:make-uuid))
           (node (make-instance 'cytoscape:element 
                                :group "nodes" 
                                :data (list (cons "id" new-id)))))
      (setf (fill-pointer stack) stack-index)
      (vector-push-extend (list (cons new-id t)) stack)
      (incf stack-index)
      (jupyter-widgets:observe node :selected
        (lambda (inst name type old-value new-value source)
          (declare (ignore inst name type old-value new-value source))
          (update-commands instance)))
      (setf (cytoscape:elements cyto)
            (append (cytoscape:elements cyto)
                    (list node))))
    (update-commands instance)))


(defun add-edge (instance id)
  (with-slots (stack-index stack cyto edge-source)
              instance
    (if edge-source
      (let* ((new-id (jupyter:make-uuid))
             (edge (make-instance 'cytoscape:element
                                  :group "edges" 
                                  :data (list (cons "id" new-id)
                                              (cons "source" edge-source)
                                              (cons "target" id)))))
        (setf (fill-pointer stack) stack-index)
        (vector-push-extend (list (cons new-id t)) stack)
        (incf stack-index)
        (jupyter-widgets:observe edge :selected
           (lambda (inst name type old-value new-value source)
            (declare (ignore inst name type old-value new-value source))
            (update-commands instance)))
        (setf (cytoscape:elements cyto)
              (append (cytoscape:elements cyto)
                      (list edge)))
        (setf edge-source nil)
        (update-commands instance))
      (setf edge-source id))))


(defun delete-node (instance id)
  (with-slots (stack-index stack cyto)
              instance
    (setf (fill-pointer stack) stack-index)
    (incf stack-index)
    (let (actions)
      (dolist (element (cytoscape:elements cyto))
        (when (or (equal id (cdr (assoc "id" (cytoscape:data element) :test #'string=)))
                  (equal id (cdr (assoc "source" (cytoscape:data element) :test #'string=)))
                  (equal id (cdr (assoc "target" (cytoscape:data element) :test #'string=))))
          (push (cons (cdr (assoc "id" (cytoscape:data element) :test #'string=)) nil) actions)
          (setf (cytoscape:removed element) t)))
      (vector-push-extend actions stack)
      (update-commands instance))))


(defun delete-edge (instance id)
  (with-slots (stack-index stack cyto)
              instance
    (setf (fill-pointer stack) stack-index)
    (vector-push-extend (list (cons id nil)) stack)
    (incf stack-index)
    (dolist (element (cytoscape:elements cyto))
      (when (equal id (cdr (assoc "id" (cytoscape:data element) :test #'string=)))
        (setf (cytoscape:removed element) t)))
    (update-commands instance)))


(defun delete-elements (instance)
  (with-slots (stack-index stack cyto)
              instance
    (setf (fill-pointer stack) stack-index)
    (incf stack-index)
    (let (actions nodes)
      (dolist (element (cytoscape:elements cyto))
        (when (cytoscape:selected element)
          (push (cons (cdr (assoc "id" (cytoscape:data element) :test #'string=)) nil) actions)
          (setf (cytoscape:removed element) t)
          (when (equal "nodes" (cytoscape:group element))
            (push (cdr (assoc "id" (cytoscape:data element) :test #'string=)) nodes))))
      (dolist (element (cytoscape:elements cyto))
        (when (or (member (cdr (assoc "source" (cytoscape:data element) :test #'string=)) nodes :test #'equal)
                  (member (cdr (assoc "target" (cytoscape:data element) :test #'string=)) nodes :test #'equal))
          (push (cons (cdr (assoc "id" (cytoscape:data element) :test #'string=)) nil) actions)
          (setf (cytoscape:removed element) t)))
      (vector-push-extend actions stack)
      (update-commands instance))))


(defun reverse-edge (instance id)
  (with-slots (stack-index stack cyto)
              instance
    (let ((new-id (jupyter:make-uuid)))
      (setf (fill-pointer stack) stack-index)
      (vector-push-extend (list (cons id nil) (cons new-id t)) stack)
      (incf stack-index)
      (dolist (element (cytoscape:elements cyto))
        (when (equal id (cdr (assoc "id" (cytoscape:data element) :test #'string=)))
          (setf (cytoscape:removed element) nil)
          (setf (cytoscape:elements cyto)
                (append (cytoscape:elements cyto)
                        (list (make-instance 'cytoscape:element :group "edges" :data (list (cons "id" new-id)
                                                                                           (cons "source" (cdr (assoc "target" (cytoscape:data element) :test #'string=)))
                                                                                           (cons "target" (cdr (assoc "source" (cytoscape:data element) :test #'string=))))))))))
      (update-commands instance))))


(defmethod initialize-instance :after ((instance editor) &rest initargs &key &allow-other-keys)
  (declare (ignore initargs))
  (with-slots (undo-command redo-command add-node-command add-edge-command delete-node-command
               delete-edge-command delete-elements-command reverse-edge-command cyto)
              instance
    (cytoscape:on-menu-command-select undo-command
                                      (lambda (command-instance id)
                                        (declare (ignore command-instance id))
                                        (undo instance)))
    (cytoscape:on-menu-command-select redo-command
                                      (lambda (command-instance id)
                                        (declare (ignore command-instance id))
                                        (redo instance)))
    (cytoscape:on-menu-command-select add-node-command
                                      (lambda (command-instance id)
                                        (declare (ignore command-instance id))
                                        (add-node instance)))
    (cytoscape:on-menu-command-select add-edge-command
                                      (lambda (command-instance id)
                                        (declare (ignore command-instance))
                                        (add-edge instance id)))
    (cytoscape:on-menu-command-select delete-node-command
                                      (lambda (command-instance id)
                                        (declare (ignore command-instance))
                                        (delete-node instance id)))
    (cytoscape:on-menu-command-select delete-edge-command
                                      (lambda (command-instance id)
                                        (declare (ignore command-instance))
                                        (delete-edge instance id)))
    (cytoscape:on-menu-command-select delete-elements-command
                                      (lambda (command-instance id)
                                        (declare (ignore command-instance id))
                                        (delete-elements instance)))
    (cytoscape:on-menu-command-select reverse-edge-command
                                      (lambda (command-instance id)
                                        (declare (ignore command-instance))
                                        (reverse-edge instance id)))
    (setf (cytoscape:context-menus cyto)
          (list (make-instance 'cytoscape:context-menu
                               :selector "core"
                               :commands (list undo-command redo-command delete-elements-command add-node-command))
                (make-instance 'cytoscape:context-menu
                               :selector "node"
                               :commands (list undo-command redo-command delete-node-command add-node-command add-edge-command))
                (make-instance 'cytoscape:context-menu
                               :selector "edge"
                               :commands (list undo-command redo-command delete-edge-command add-node-command reverse-edge-command))))))

(defparameter ce (make-instance 'editor))

(cyto ce)

#<The STANDARD-CLASS COMMON-LISP-USER::EDITOR>

UPDATE-COMMANDS

UNDO

REDO

ADD-NODE

...

DELETE-ELEMENTS

REVERSE-EDGE

#<STANDARD-METHOD INITIALIZE-INSTANCE :AFTER (EDITOR)>

CE

A Jupyter Widget